In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from math import atan2, pi, sqrt, cos, sin, floor
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten, LSTM, Bidirectional, concatenate
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform
from keras_flops import get_flops
import pickle
import csv
import random
import itertools
import math
import time
import sys
sys.path.insert(0, '../dataset2/')
from data_utils_2 import *
from ionet_traj_utils import *

## Import Dataset

In [ ]:
window_size = 100
stride = 20

f = '../dataset2/'
X_train,Y_Pos_train, Physics_Vec_train, x_vel_train, y_vel_train, x0_list_train, y0_list_train, size_of_each_train = import_agrobot_dataset_p3(dataset_folder=f, type_flag=1, window_size=window_size, stride=stride)
P = np.repeat(Physics_Vec_train,window_size).reshape((Physics_Vec_train.shape[0],window_size,1))
X_train = np.concatenate((X_train,P),axis=2)

X_test,Y_Pos_test, Physics_Vec_test, x_vel_test, y_vel_test, x0_list_test, y0_list_test, size_of_each_test= import_agrobot_dataset_p3(type_flag = 2, dataset_folder=f,window_size=window_size, stride=stride)
P_test = np.repeat(Physics_Vec_test,window_size).reshape((Physics_Vec_test.shape[0],window_size,1))
X_test = np.concatenate((X_test,P_test),axis=2)

## Generate Heading, Displacement and Timestamp Intervals

In [ ]:
disp_train = np.sqrt((x_vel_train**2) + (y_vel_train**2)) 
disp_test = np.sqrt((x_vel_test**2) + (y_vel_test**2)) 

In [ ]:
heading_train = np.zeros((Y_Pos_train.shape[0]))
prev = 0
for i in range(Y_Pos_train.shape[0]):
    theta = abs_heading(Y_Pos_train[i,-1,0],Y_Pos_train[i,-1,1],Y_Pos_train[i,0,0],Y_Pos_train[i,0,1])
    if theta<180:
        theta = theta + 180

    heading_train[i] = theta - prev
    if(heading_train[i]>100 or heading_train[i]<-100):
        theta2 = theta
        prev2 = prev
        if theta<prev:
            theta2 = theta + 360
        else:
            prev2 =  prev + 360
        heading_train[i] = theta2 - prev2 
    prev = theta

In [ ]:
heading_test = np.zeros((Y_Pos_test.shape[0]))
prev = 0
for i in range(Y_Pos_test.shape[0]):
    theta = abs_heading(Y_Pos_test[i,-1,0],Y_Pos_test[i,-1,1],Y_Pos_test[i,0,0],Y_Pos_test[i,0,1])
    if theta<180:
        theta = theta + 180

    heading_test[i] = theta - prev
    if(heading_test[i]>100 or heading_test[i]<-100):
        theta2 = theta
        prev2 = prev
        if theta<prev:
            theta2 = theta + 360
        else:
            prev2 =  prev + 360
        heading_test[i] = theta2 - prev2 
    prev = theta

## Model Training

In [ ]:
i = Input(shape=(window_size, X_train[:,:,0:6].shape[2]))
f_layer = LSTM(128,dropout=0.25,return_sequences=True)
b_layer = LSTM(128,dropout=0.25,return_sequences=True,go_backwards=True)
k = Bidirectional(f_layer,backward_layer=b_layer)(i)
output1 = Dense(1, activation='linear', name='disp')(k)
output2 = Dense(1, activation='linear', name='head')(k)
model = Model(inputs=[i],outputs=[output1, output2])
model.summary()

In [ ]:
model_name = 'ionet_dset2.hdf5'
opt = tf.keras.optimizers.Adam(learning_rate=0.0015)
model.compile(loss={'disp': 'mse','head':'mse'},optimizer=opt)
checkpoint = ModelCheckpoint(model_name, monitor='loss', verbose=1, save_best_only=True)
model.fit(x=X_train[:,:,0:6],y=[disp_train,heading_train*0.0174533],
          callbacks=[checkpoint],epochs=1500,batch_size=32,verbose=1,shuffle=True)

## Evaluation

In [ ]:
#model_name = 'ionet_dset2.hdf5'
model = load_model(model_name)

Unseen Trajectory

In [ ]:
ATE = []
RTE = []
ATE_dist = []
RTE_dist = []
for i in range(len(size_of_each_test)):
    Pvx, Pvy = ionet_pos_generator(X_test[:,:,0:6], size_of_each_test, 
                   x0_list_test, y0_list_test, window_size, stride,i,model)   
    Gvx, Gvy = ionet_GT_pos_generator(disp_test,heading_test,size_of_each_test,
                                x0_list_test, y0_list_test, window_size, stride,i)
    
    at, rt, at_all, rt_all = Cal_TE(Gvx, Gvy, Pvx, Pvy,
                                    sampling_rate=100,window_size=window_size,stride=stride)
    ATE.append(at)
    RTE.append(rt)
    ATE_dist.append(Cal_len_meters(Gvx, Gvy))
    RTE_dist.append(Cal_len_meters(Gvx, Gvy, 600))
    print('ATE, RTE, Trajectory Length, Trajectory Length (60 seconds)',ATE[i],RTE[i],ATE_dist[i],RTE_dist[i])
    
print('Median ATE and RTE', np.median(ATE),np.median(RTE))
print('Mean ATE and RTE', np.mean(ATE),np.mean(RTE))
print('STD ATE and RTE', np.std(ATE),np.std(RTE))

Seen Trajectory

In [ ]:
ATE = []
RTE = []
ATE_dist = []
RTE_dist = []
for i in range(len(size_of_each_train)):
    Pvx, Pvy = ionet_pos_generator(X_train[:,:,0:6], size_of_each_train, 
                   x0_list_train, y0_list_train, window_size, stride,i,model)   
    Gvx, Gvy = ionet_GT_pos_generator(disp_train,heading_train,size_of_each_train,
                                x0_list_train, y0_list_train, window_size, stride,i)
    
    at, rt, at_all, rt_all = Cal_TE(Gvx, Gvy, Pvx, Pvy,
                                    sampling_rate=100,window_size=window_size,stride=stride)
    ATE.append(at)
    RTE.append(rt)
    ATE_dist.append(Cal_len_meters(Gvx, Gvy))
    RTE_dist.append(Cal_len_meters(Gvx, Gvy, 600))
    print('ATE, RTE, Trajectory Length, Trajectory Length (60 seconds)',ATE[i],RTE[i],ATE_dist[i],RTE_dist[i])
    
print('Median ATE and RTE', np.median(ATE),np.median(RTE))
print('Mean ATE and RTE', np.mean(ATE),np.mean(RTE))
print('STD ATE and RTE', np.std(ATE),np.std(RTE))